# CoRecommender - Usage Example 
A flexible collaborative filtering recommendation system that supports both item-based and user-based approaches using nearest neighbors with cosine similarity. Built with Python, NumPy, Pandas, and scikit-learn.

In [ ]:
import numpy as np
import pandas as pd
from co_recommender import CoRecommender

## Read Data Sources

the data must contains:
1. `uid`: user id, how made the interaction
2. `iid`: item id, that the interaction made on it.
3. `dynamic`: this user free column name, act as indicator of the interaction and it's scale means how mush user interacts with this item.

`dynamic` means, you can name it whatever you want, it could be anything, but it should be Numeric Value.
it could be for example rate, num_of_visits, likes, num_of_clicks...

#### Read it from CSV

In [3]:
users_interaction_data = pd.read_csv(
    'database/ratings.csv',
    usecols=['userId', 'movieId', 'rating'],
).rename(
    columns={
        'userId': 'uid',
        'movieId': 'iid',
        'rating': 'rate'
    })

movies = pd.read_csv(
    'database/movies.csv',
    index_col='movieId',
    converters={"genres": lambda x: ', '.join(sorted(x.split('|')))})

#### Read it from JSON

In [29]:
import json
with open('database/users_interaction_data_sample.json', 'r') as f:
    users_interaction_data_json = f.read()
    users_interaction_data = json.loads(users_interaction_data_json)
    users_interaction_data = pd.DataFrame(users_interaction_data)

In [30]:
print(users_interaction_data_json)

[
    {
        "uid": 1,
        "iid": 1,
        "rate": 4.0
    },
    {
        "uid": 1,
        "iid": 3,
        "rate": 4.0
    }
]


#### Preview

In [23]:
users_interaction_data

,uid,iid,rate
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [5]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),"Adventure, Animation, Children, Comedy, Fantasy"
2,Jumanji (1995),"Adventure, Children, Fantasy"
3,Grumpier Old Men (1995),"Comedy, Romance"
4,Waiting to Exhale (1995),"Comedy, Drama, Romance"
5,Father of the Bride Part II (1995),Comedy


## Create CoRecommender
you need to specify
1. `mode`: the tool handle two recommendations types, item-based and user-based, you should choose one type at once, <br> `user` -> user-based <br> `item` -> item-based.
2. `indic`: indication column name, `dynamic` colname like above, ex. rate. 

In [6]:
recommendation_type = 'item'
interaction_indication_col = 'rate'

rec_sys = CoRecommender(
    mode=recommendation_type,
    indic=interaction_indication_col,
)

### Train The model
Means Convert the user-item interaction data into suitable formate and save it based on recommendation Type

In [7]:
rec_sys.train_model(users_interaction_data)

### Running Sample

In [8]:
# get unique users and items
uids = users_interaction_data['uid'].unique()
rec_data = users_interaction_data

In [9]:
# get random user id for testing the model.
user_id = np.random.choice(uids)

# get all data for a specific user only.
user_preferred_products = rec_data[
    rec_data['uid'] == user_id][['iid', 'rate']]

user_preferred_products = user_preferred_products.sort_values(
    by='rate', ascending=False).reset_index(drop=True)

print(f'User id: {user_id}, Preferred Items:')
user_preferred_products.head()

User id: 343, Preferred Items:


,iid,rate
0,527,5.0
1,4226,5.0
2,1213,5.0
3,2571,5.0
4,2959,5.0


### Get Recommendations

#### Parameters
1. `user_id`: that you want to get recommendation for him, and based on his behavior. `required` for user-based.
2. `user_prev_data`: user previous interaction data, that based on it get recommendations. `required` for item-based
3. `n_recommendations`: max number of recommendations items you want to get `required`.
4. `n_similar_entities`: for each item or user how much you wan to get similar items. `required`
5. `print_results`: print logs, `optional` with defaults `True`

#### Returns
in Case User-Based
1. `recommended_items_ids`: list of all recommended items ids.
2. `similar_users_id`:  list of all similar users ids.

in Case Item-Based
1. `recommended_items_ids`: list of all recommended items ids.
2. `relative_recommendations`: this dictionary that contains the user preferred item id as key, and similar items ids that recommended based on this key.

In [ ]:
# NOTE: you can't call or use this method before train the mode at least one time.
recommendations = rec_sys.recommend_items(
    user_id=user_id,  # user id to recommend items for.
    user_prev_data=user_preferred_products,  # user previous interactions.
    n_recommendations=200,  # number of items to recommend.
    n_similar_entities=10,  # number of similar items or users to get.
    print_results=False  # set to True if you want to print the results.
)

### Show Relative Recommendations Results (Item-based)

In [11]:
if recommendation_type == 'item':
    user_watched_movies = set(user_preferred_products['iid'].values)
    print("Relative Recommendations:")

    relative_recs = recommendations['relative_recommendations']

    printed_mids = set()

    for i, (r_mid, r_mids) in enumerate(relative_recs.items(), 1):

        print(f'{i}. because user loved ->', ' - '.join(
            movies.loc[r_mid][['title', 'genres']].values))

        e = 0
        for m_id in r_mids:
            if m_id in user_watched_movies or m_id in printed_mids:
                continue

            printed_mids.add(m_id)
            e = e+1
            print(f'   {e}.', ' - '.join(
                movies.loc[m_id][['title', 'genres']].values))

        print('-'*100)

Relative Recommendations:
1. because user loved -> Schindler's List (1993) - Drama, War
   1. Forrest Gump (1994) - Comedy, Drama, Romance, War
   2. Saving Private Ryan (1998) - Action, Drama, War
   3. Braveheart (1995) - Action, Drama, War
   4. Sixth Sense, The (1999) - Drama, Horror, Mystery
   5. Seven (a.k.a. Se7en) (1995) - Mystery, Thriller
   6. Apollo 13 (1995) - Adventure, Drama, IMAX
   7. Jurassic Park (1993) - Action, Adventure, Sci-Fi, Thriller
   8. Fugitive, The (1993) - Thriller
   9. Speed (1994) - Action, Romance, Thriller
   10. Green Mile, The (1999) - Crime, Drama
   11. Dances with Wolves (1990) - Adventure, Drama, Western
   12. Fargo (1996) - Comedy, Crime, Drama, Thriller
----------------------------------------------------------------------------------------------------
2. because user loved -> Memento (2000) - Mystery, Thriller
   1. Eternal Sunshine of the Spotless Mind (2004) - Drama, Romance, Sci-Fi
   2. Kill Bill: Vol. 1 (2003) - Action, Crime, Thrill

### User Preferred Items 

In [12]:
print("Top 10 movies rated by user: ")
top_10_movies = user_preferred_products.sort_values(
    'rate', ascending=False).head(10)

movies.merge(top_10_movies, left_index=True, right_on='iid')[
    ['title', 'genres', 'rate']].reset_index(drop=True)

Top 10 movies rated by user: 


,title,genres,rate
0,Schindler's List (1993),"Drama, War",5.0
1,Goodfellas (1990),"Crime, Drama",5.0
2,"Matrix, The (1999)","Action, Sci-Fi, Thriller",5.0
3,Fight Club (1999),"Action, Crime, Drama, Thriller",5.0
4,Memento (2000),"Mystery, Thriller",5.0
5,Grave of the Fireflies (Hotaru no haka) (1988),"Animation, Drama, War",5.0
6,"Lord of the Rings: The Two Towers, The (2002)","Adventure, Fantasy",5.0
7,City of God (Cidade de Deus) (2002),"Action, Adventure, Crime, Drama, Thriller",5.0
8,Infernal Affairs (Mou gaan dou) (2002),"Crime, Drama, Thriller",5.0
9,Superbad (2007),Comedy,5.0


### Recommended Items

In [13]:
print("Recommended Movies:")

rec_movies = movies.loc[recommendations['recommended_items_ids']]
rec_movies.head(10)

Recommended Movies:


,title,genres
movieId,,
356,Forrest Gump (1994),"Comedy, Drama, Romance, War"
61024,Pineapple Express (2008),"Action, Comedy, Crime"
56156,Hitman (2007),"Action, Crime, Thriller"
6433,"Man with the Movie Camera, The (Chelovek s kin...",Documentary
2028,Saving Private Ryan (1998),"Action, Drama, War"
1210,Star Wars: Episode VI - Return of the Jedi (1983),"Action, Adventure, Sci-Fi"
148671,Saw (2003),"Crime, Horror"
6713,Millennium Actress (Sennen joyû) (2001),"Animation, Drama, Romance"
110,Braveheart (1995),"Action, Drama, War"
